## Interface 

In this notebook we automatise the computation of matrix elements for later use by a Krylov iterative solver. We save all symbolic results in a file containing reusable functions:

    interfaceME.py
    
This file only has to be generated once (per Green's function. 

In [1]:
from sympy import *
import numpy as np
import re

### Definitions and auxiliary functions

In [2]:
x1, y1, z1, x2, y2, z2 = symbols("x1, y1, z1, x2, y2, z2", real=True)
b, h, eta = symbols("b, h, eta", real=True, positive=True)
theta = symbols("theta", real=True) ##eventually rename theta->psi


mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma = symbols(
    "mu, nu1, nu2, gamma2, sigma, eta1, eta2, beta, alpha, tau, gamma1, kappa1, kappa2, gamma", integer=True)

R1 = Array([x1,y1,z1]) #why not use matrix here?
R2 = Array([x2,y2,z2])

KroneckerDelta = eye(3)

M  = Array([[1,0,0],[0,1,0],[0,0,-1]]) #diag(1,1,-1)

Mf = Array([[(1-theta)/(1+theta),0,0], [0, (1-theta)/(1+theta),0], [0,0,-1]])
#diag( (1-theta)/(1+theta), (1-theta)/(1+theta), -1) 

R2M = tensorcontraction(tensorproduct(M,R2), (1,2))

rM = R1 - R2M

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)#sqrt(summation(r[alpha]**2,(alpha,0,2)))

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Laplacian(G, r):
    return diff(G, r[0], r[0]) + diff(G, r[1], r[1]) + diff(G, r[2], r[2])


## for some reason I cannot sum over 'symbols' here whenever rM is a Matrix, not an Array. Who knows why, but this seems to work. 

def Curl(mu, beta): ##Curl is always with respect to R1
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(mu,i,j)*diff(G(j,beta),R1[i]) ## Need dependence of G on rM and R2? 
    return result

def tilde_Curl(alpha, sigma): ##tilde_Curl is always with respect to R2
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(sigma,i,j)*diff(G(alpha, j),R2[i])
    return result

def CurlCurl(mu, sigma):
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(mu, i, j)*eps(sigma, k, l)*diff(G(j, l), R1[i], R2[k])
    return result

In [4]:
def Fl(G, l):
    return G + b**2/(4*l+6)*Laplacian(G, R1)

def tilde_Fl(G,l):
    return G + b**2/(4*l+6)*Laplacian(G, R2)

def FlFl(G, l, lprime):
    return G + b**2/(4*l+6)*Laplacian(G, R1) + b**2/(4*lprime+6)*Laplacian(G, R2) + b**4/((4*l+6)*(4*lprime+6))*Laplacian(Laplacian(G, R1), R2)

In [5]:
## Green's function
##first G* part, which is the same for ii and ij, later G0 part for ii

def GO(r, alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(r) + r[alpha]*r[beta]/norm(r)**3) ##can reuse for particle-particle interactions

## only for Gstar, particle - image interactions
def Stokeslet(alpha, beta):
    result = 0
    for i in range(3):
        result += Mf[beta, i]*GO(rM, alpha, i)
    return result

def Dipole(alpha, beta):
    result = 0
    for i in range(3):
        result += -2*R2[2]*theta/(1+theta)*diff(GO(rM, alpha, 2) * M[beta, i], R1[i])
    return result

def Quadrupole(alpha, beta):
    result = 0
    for i in range(3):
        result += R2[2]**2*theta/(1+theta)*Laplacian(GO(rM, alpha, i) * M[beta, i], R1)
    return result

def G(alpha, beta): #introduce flag to switch between G0 and Gstar, for whichever we have to compute the matrix elements
    return Stokeslet(alpha, beta) + Dipole(alpha, beta) + Quadrupole(alpha, beta)

In [6]:
## is there not a smarter way to do this? No dependence on rM and R2.. in python want xij, yij etc not x1- x2
## Need: self-interaction and ineraction with image. Interaction with other particle and with its image
## constant, R1=R2 (also constant), rij, and also rij. How to compute all of them with same code? 

## keep dependence on rM and R2: 
        ## for self-interaction choose rM = 0, 0, 2h and R2= 0,0,h
        ## for interaction choose: x1-x2=xij with particle, z1+z2 = ?

## keep dependence for now, might give greater flexibility later - no, delete to make shorter and write extra functions for particle-particle interactions
#for interactions, replace rM by xij, yij, dz = z1 + z2 or so, but there is also zij for particle-particle interactions, need another Greens function GO(rij, alpha, beta)

## could use flag in G, whether to return Go or Gstar

In [14]:
def G1s1s(alpha, beta):
    return FlFl( G(alpha, beta), 0, 0)

def G2a1s(mu, beta):
    return b*tilde_Fl(Curl(mu, beta), 0)

def G1s2a(alpha,sigma):
    return 0.5*b*Fl(tilde_Curl(alpha, sigma), 0)

def G2a2a(mu, sigma):
    return 0.5*b**2*CurlCurl(mu, sigma)

def G1s2s(alpha, sigma, eta1):
    return 0.5*b*FlFl((diff(G(alpha,sigma), R2[eta1]) + diff(G(alpha, eta1), R2[sigma])), 0, 1)

def G2a2s(mu, sigma, eta1):
    return 0.5*b**2*tilde_Fl( (diff(Curl(mu,sigma), R2[eta1]) + diff(Curl(mu, eta1), R2[sigma])), 1)

def G2s1s(mu, nu1, beta):
    return 0.5*b*FlFl( diff( G(mu, beta), R1[nu1]) + diff( G(nu1, beta), R1[mu]), 1,0)

def G2s2a(mu, nu1, sigma):
    return 0.25*b**2*Fl( (diff(tildeCurl(mu,sigma), R1[nu1]) + diff(tildeCurl(nu1, sigma), R1[mu])), 1)

def G2s2s(mu, nu1, sigma, eta1):
    return 0.25*b**2*FlFl( diff( G(mu,sigma), R1[nu1], R2[eta1]) + diff( G(nu1,sigma), R1[mu], R2[eta1])
                          +  diff( G(mu,eta1), R1[nu1], R2[sigma]) + diff( G(nu1,eta1), R1[mu], R2[sigma]), 1,1)

def G2s3t(mu, nu1, eta2):
    return -0.1*b**3*Fl( Laplacian( diff( G(mu, eta2), R1[nu1]) + diff( G(nu1, eta2), R1[mu]), R2), 1)

def G2s3a(mu, nu1, sigma, eta2):
    return 1/3*b**3*Fl( diff( diff( tildeCurl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(mu, beta), R2), (beta,0,2)), R1[nu1]) 
                       + diff( diff( tildeCurl(nu1, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( G(nu1, beta), R2), (beta,0,2)), R1[mu]), 1)

def G3a2s(mu, nu2, sigma, nu1):
    return 0.25*b**3*tilde_Fl( diff( diff( Curl(tau, beta), R2[kappa1]) + diff( Curl(tau, kappa1), R2[beta]), R1[gamma2])
                              + diff( diff( Curl(gamma2, beta), R2[kappa1]) + diff( Curl(gamma2, kappa1), R2[beta]), R1[tau]), 1)

def G3a3a(mu, nu2, sigma, eta2):
    return 1/3*b**4*( diff( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(mu, beta), R2), (beta,0,2)), R1[nu2]) 
                       + diff( diff( CurlCurl(nu2, sigma), R2[eta2]) - 1/3*summation( eps(sigma, eta2, beta)*Laplacian( Curl(nu2, beta), R2), (beta,0,2)), R1[mu]) )

def G1s3t(alpha, eta2):
    return -0.2*b**2*Fl( Laplacian( G(alpha, eta2), R2), 0)

def G1s3a(alpha, sigma, eta2):
    return 2/3*b**2*Fl( diff( tildeCurl(alpha, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( G(alpha,beta), R2), (beta,0,2)), 0)

def G2a3a(mu, sigma, eta2):
    return 2/3*b**3*( diff( CurlCurl(mu, sigma), R2[eta2]) - 1/3*summation(eps(sigma, eta2, beta)*Laplacian( Curl(mu,beta), R2), (beta,0,2)) )

def G3a1s(mu, nu2, beta):
    return 0.5*b**2*tilde_Fl( diff( Curl(mu,beta), R1[nu2]) + diff( Curl(nu2,beta), R1[mu]), 0)



In [8]:
## Really slow, can return to use 'summation' now that rM is an Array? Would that be quicker? instead of silly for loop
## Nope, for loop is at least as fast 

In [9]:
## Need Curl argument G? Def need it in Laplacian and Fl, so it is flexible. But Curl never taken of different function. Curl always only taken of G directly. 

In [10]:
%%time
bla = G2a2s(0, 1, 2)

CPU times: user 25.6 s, sys: 58.4 ms, total: 25.7 s
Wall time: 25.7 s


In [11]:
simplify(bla.subs({x1:0, y1:0, z1:h, x2:0, y2:0, z2:h}))

b**2*(0.0375*b**2*theta - 0.046875*h**2*(theta + 1))/(pi*eta*h**5*(theta + 1))